In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
import datasets

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
dataset = datasets.load_dataset("alpindale/light-novels")

README.md:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

LightNovels.txt:   0%|          | 0.00/907M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9240994 [00:00<?, ? examples/s]

In [3]:
text_corpus = "\n".join(example["text"] for example in dataset["train"])

In [4]:
corpus_size = len(text_corpus)
print(corpus_size)

879870486


In [5]:
train_size = int(100*1e6) # 100 million
val_size = int(10*1e6) # 10 million

In [6]:
text = text_corpus[:train_size + val_size]

In [7]:
train_text = text[:train_size]
val_text = text[train_size:]

In [8]:
print(train_text[:1000])


 
“RISHE IRMGARD WEITZNER! You are a vile woman! A truly malicious creature unworthy of the crown prince! As of this instant, our engagement is off!” 
“As you wish, Your Highness.”
“…Huh?”
The prince’s declaration echoed throughout the ballroom. Rishe, the duke’s daughter, bowed in response, her coral-toned hair swaying gracefully. The assembled guests found themselves captivated by her dignified beauty. Despite having just witnessed a broken engagement, they couldn’t bring themselves to pity her.
Slowly, Rishe lifted her head and fixed the prince with a pale-emerald gaze. It caught him off guard, but he quickly recovered.
“Didn’t you hear what I just said?! I’m calling off our engagement! Doesn’t that trouble you?”
“Not really.”
Rishe already knew what happened next: false accusations, exile, and her family severing all ties and leaving her to survive on her own. She turned from the prince. There was no point wasting any time.
This is my seventh loop, after all.
Rishe had done this a

In [9]:
print(val_text[:1000])

shining even brighter, and I instinctively knew that if I didn’t stop her, we would be here all day.
      I shot Gerard a look. “Uh, so...Grandpa Gerard said he wants to listen to every last detail of your struggles and how you overcame them, but at a later time. Can you just tell us what this gate does for now?”
      “Oh, really? Okay!” Shutola gave Gerard a big smile. “Guess we’re saving the best part for last, right, grandpa?”
      “We sure are!” the dark knight immediately replied in a very loud voice.
      Today is the first time I’ve seen the phrase “the right person for the right job” fit a situation so perfectly. I’m sure Gerard would be more than happy to listen to his grandchild lecturing for hours, and Shutola would be happy to have an audience that she can share all the results of her latest research with. This is exactly what a mutually beneficial relationship is.
      Shutola continued. “I’ll simplify things, then. As I said, the amount of MP needed to activate this 

In [10]:
# Couting the ocurrences of each character in the text
char_count = {}
for c in text:
    char_count[c] = char_count.get(c, 0) + 1

In [11]:
chars_sorted = sorted(char_count, key=lambda c : char_count[c], reverse=True)
for c in chars_sorted:
    print(f"char: {c}, count: {char_count[c]}")

char:  , count: 20277720
char: e, count: 10274375
char: t, count: 7476137
char: a, count: 6556039
char: o, count: 6159843
char: n, count: 5391861
char: i, count: 5313162
char: s, count: 4959160
char: h, count: 4913132
char: r, count: 4701548
char: d, count: 3507731
char: l, count: 3469492
char: u, count: 2613175
char: m, count: 2018351
char: c, count: 1885387
char: g, count: 1881116
char: w, count: 1827437
char: y, count: 1723450
char: f, count: 1575931
char: ., count: 1409342
char: p, count: 1395277
char: b, count: 1188933
char: ,, count: 1126783
char: 
, count: 1066867
char: k, count: 907004
char: v, count: 724144
char: I, count: 587763
char: ’, count: 544616
char: “, count: 409759
char: ”, count: 408253
char: T, count: 311392
char: A, count: 242200
char: S, count: 212978
char: H, count: 186999
char: ?, count: 185867
char: W, count: 140171
char: !, count: 135123
char: x, count: 134771
char: M, count: 123304
char: -, count: 123191
char: B, count: 114465
char: j, count: 113587
char: C,

In [24]:
min_count = 250
vocab = []
for c in char_count:
    if char_count[c] >= min_count:
        vocab.append(c)

unk_token = "<UNK>"
vocab.append(unk_token)
vocab_size = len(vocab)
print(vocab)
print(vocab_size)

['\n', ' ', '“', 'R', 'I', 'S', 'H', 'E', 'M', 'G', 'A', 'D', 'W', 'T', 'Z', 'N', '!', 'Y', 'o', 'u', 'a', 'r', 'e', 'v', 'i', 'l', 'w', 'm', 'n', 't', 'y', 'c', 's', 'h', 'f', 'p', ',', 'g', '”', '.', '…', '?', '’', 'd', 'b', 'k', '-', 'j', 'x', 'z', 'q', ':', '—', 'é', 'L', 'F', '*', 'B', 'O', 'U', ';', 'C', 'P', '‘', 'K', 'J', '(', '2', ')', '0', '7', 'V', '8', '3', 'Q', '9', '4', '1', '5', '{', '}', '%', '6', '\xa0', '#', 'ö', 'ü', '–', 'è', '\t', 'X', '©', '|', '[', ']', '+', 'ō', '~', 'ï', "'", '_', '\u200b', '\xad', '♪', '"', '■', '＊', '✧', '«', '»', '〈', '〉', '\u3000', '─', '◇', '【', '】', '†', '<UNK>']
119


In [25]:
i_to_char = {i : c for i, c in enumerate(vocab)}
char_to_i = {c: i for i, c in i_to_char.items()}
encode = lambda s : [char_to_i.get(c, char_to_i[unk_token]) for c in s]
decode = lambda t : "".join([i_to_char[i] for i in t])
encoded = encode("Hello thereω")
decoded = decode(encoded)
print(encoded)
print(decoded)

[6, 22, 25, 25, 18, 1, 29, 33, 22, 21, 22, 118]
Hello there<UNK>


In [26]:
print(char_to_i)

{'\n': 0, ' ': 1, '“': 2, 'R': 3, 'I': 4, 'S': 5, 'H': 6, 'E': 7, 'M': 8, 'G': 9, 'A': 10, 'D': 11, 'W': 12, 'T': 13, 'Z': 14, 'N': 15, '!': 16, 'Y': 17, 'o': 18, 'u': 19, 'a': 20, 'r': 21, 'e': 22, 'v': 23, 'i': 24, 'l': 25, 'w': 26, 'm': 27, 'n': 28, 't': 29, 'y': 30, 'c': 31, 's': 32, 'h': 33, 'f': 34, 'p': 35, ',': 36, 'g': 37, '”': 38, '.': 39, '…': 40, '?': 41, '’': 42, 'd': 43, 'b': 44, 'k': 45, '-': 46, 'j': 47, 'x': 48, 'z': 49, 'q': 50, ':': 51, '—': 52, 'é': 53, 'L': 54, 'F': 55, '*': 56, 'B': 57, 'O': 58, 'U': 59, ';': 60, 'C': 61, 'P': 62, '‘': 63, 'K': 64, 'J': 65, '(': 66, '2': 67, ')': 68, '0': 69, '7': 70, 'V': 71, '8': 72, '3': 73, 'Q': 74, '9': 75, '4': 76, '1': 77, '5': 78, '{': 79, '}': 80, '%': 81, '6': 82, '\xa0': 83, '#': 84, 'ö': 85, 'ü': 86, '–': 87, 'è': 88, '\t': 89, 'X': 90, '©': 91, '|': 92, '[': 93, ']': 94, '+': 95, 'ō': 96, '~': 97, 'ï': 98, "'": 99, '_': 100, '\u200b': 101, '\xad': 102, '♪': 103, '"': 104, '■': 105, '＊': 106, '✧': 107, '«': 108, '»': 1

## Transformer Language model

In [37]:
class AttentionHead(nn.Module):
    def __init__(self, fin, head_size):
        super().__init__()
        self.head_size = head_size
        self.key = nn.Linear(fin, head_size, bias=False)
        self.query = nn.Linear(fin, head_size, bias=False)
        self.value = nn.Linear(fin, head_size, bias=False)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        
        lower_triangular = torch.tril(torch.ones(T, T, device=device))
        wei = (q @ k.transpose(-2, -1)) * (self.head_size ** -0.5) # Scaling
        wei = self.dropout(wei)
        wei = wei.masked_fill(lower_triangular == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        out = wei @ v # (B, T, head_size)
        return out

In [38]:
class MultiHeadAttention(nn.Module):
    def __init__(self, fin):
        super().__init__()
        head_size = fin // n_heads
        self.heads = nn.ModuleList([AttentionHead(fin, head_size) for _ in range(n_heads)])
        self.linear = nn.Linear(fin, fin)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        outs = torch.cat([head(x) for head in self.heads], dim=-1)
        outs = self.linear(outs)
        outs = self.dropout(outs)
        return outs

In [40]:
class FeedForward(nn.Module):
    def __init__(self, fin):
        super().__init__()
        self.f = nn.Sequential(
            nn.Linear(fin, 4*fin),
            nn.ReLU(),
            nn.Linear(4*fin, fin),
            nn.Dropout(dropout_prob),
        )

    def forward(self, x):
        return self.f(x)

In [39]:
class TransformerBlock(nn.Module):
    def __init__(self, fin):
        super().__init__()
        self.mha = MultiHeadAttention(fin)
        self.ff = FeedForward(fin)
        self.ln1 = nn.LayerNorm(fin)
        self.ln2 = nn.LayerNorm(fin)

    def forward(self, x):
        x = x + self.mha(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

In [41]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_emb)
        self.positional_embedding = nn.Embedding(block_size, n_emb)
        self.transformer_blocks = nn.Sequential(*[TransformerBlock(n_emb) for _ in range(n_blocks)])
        self.linear = nn.Linear(n_emb, vocab_size)

    def forward(self, inputs, targets=None):
        token_emb = self.token_embedding(inputs)
        pos_emb = self.positional_embedding(torch.arange(inputs.shape[1], device=device))
        x = token_emb + pos_emb
        out = self.transformer_blocks(x)
        logits = self.linear(out)
        loss = None
        if targets is not None:
            logits = logits.reshape(-1, logits.shape[-1])
            targets = targets.reshape(-1)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, pred, n_tokens, block_size):
        for _ in range(n_tokens):
            block = pred[:, -block_size:]                      # (B, T)
            logits, _ = self(block)                            # (B, T, vocab_size)
            current_logits = logits[:, -1, :]                  # (B, vocab_size)
            probs = F.softmax(current_logits, dim=-1)          # (B, vocab_size)
            idx_pred = torch.multinomial(probs, num_samples=1) # (B, 1)
            pred = torch.cat([pred, idx_pred], dim=-1)         # (B, T + 1)
        return pred

In [42]:
def get_batch(tensor, batch_size, n_tokens):
    mx_size = len(tensor) - n_tokens - 1
    ix = torch.randint(low=0, high=mx_size, size=(batch_size,))
    x = torch.stack([tensor[i:i+n_tokens] for i in ix])
    y = torch.stack([tensor[i+1:i+n_tokens+1] for i in ix])
    
    assert x.shape == (batch_size, n_tokens)
    assert y.shape == (batch_size, n_tokens)
    return x, y

In [43]:
@torch.no_grad()
def evaluate_loss(tensor, eval_iterations):
    losses = torch.zeros(eval_iterations, device=device)
    for i in range(eval_iterations):
        x, y = get_batch(tensor, batch_size, block_size)
        x, y = x.to(device), y.to(device)
        _, loss = model(x, y)
        losses[i] = loss

    return losses.mean()

In [86]:
training_steps = 5000
learning_rate = 3e-4

batch_size = 32
block_size = 512
n_heads = 8
n_blocks = 6
n_emb = n_blocks * 64
dropout_prob = 0.25

In [87]:
model = Model()
model.to(device)
n_parameters = sum([p.numel() for p in model.parameters()])
print("Number of parameters:", n_parameters)

Number of parameters: 10927991


In [88]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [90]:
train_tensor = torch.tensor(encode(train_text), dtype=torch.int64)
val_tensor = torch.tensor(encode(val_text), dtype=torch.int64)

In [91]:
for i in range(training_steps):
    x, y = get_batch(train_tensor, batch_size, block_size)
    x, y = x.to(device), y.to(device)
    logits, loss = model(x, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i % 500 == 0 or i == training_steps - 1:
        train_loss = evaluate_loss(train_tensor, 100).item()
        val_loss = evaluate_loss(val_tensor, 100).item()
        print("Iteration:", i)
        print("Training loss:", train_loss)
        print("Validation loss:", val_loss)
        print("--------------------")

Iteration: 0
Training loss: 4.4150919914245605
Validation loss: 4.456362724304199
--------------------
Iteration: 500
Training loss: 2.4346230030059814
Validation loss: 2.478677272796631
--------------------
Iteration: 1000
Training loss: 2.00752854347229
Validation loss: 2.0601422786712646
--------------------
Iteration: 1500
Training loss: 1.7657358646392822
Validation loss: 1.8190914392471313
--------------------
Iteration: 2000
Training loss: 1.6245266199111938
Validation loss: 1.6965432167053223
--------------------
Iteration: 2500
Training loss: 1.529032588005066
Validation loss: 1.6127657890319824
--------------------
Iteration: 3000
Training loss: 1.4704316854476929
Validation loss: 1.551788568496704
--------------------
Iteration: 3500
Training loss: 1.4182647466659546
Validation loss: 1.5161885023117065
--------------------
Iteration: 4000
Training loss: 1.3761156797409058
Validation loss: 1.4743188619613647
--------------------
Iteration: 4500
Training loss: 1.34931063652038

In [99]:
newline_char = char_to_i["\n"]
start_token = torch.tensor([[newline_char]], dtype=torch.int64, device=device)
pred = model.generate(start_token, 3000, block_size)

In [100]:
print(decode(pred.tolist()[0]))


  “So, Vol. Sakuro of than there, as I grunse.” that if well. “I’m surteen it after a long that it, probably seems are honest.”

      Gient said.. be as if it reciving here? I didn’t need to her need?

      “This is all attentiors ontoward, are yank offit. I seek another that rewark.”

      “Heh? Sorry, that’s more.”
      A glus researches the agerisment condive seven protect that had resences. “Get offer squealing?” Moltalley over to drive anything. “I’m that Melfing for him.”

     “Yeah. on sure,” Haruyuki tone lose of Melta said. “The prisison. Commoned as royal my own now my clothiage,” said, with us heaving mater for the citizing.

      “... I then girls not you atting not jaws something, I think you’ve been away exited to plus? And comine its finish smile.”

      Soma blump.

      “Mm ‘Grand Monizaby., shouse I returned teeth usiness to rail a blizzer and someone,” Regentaro mannery had for orc. So made I’ve now that worma. Lothou but her asserably probably since would g